In [1]:
!wget https://github.com/sinanuozdemir/quick-start-guide-to-llms/raw/refs/heads/main/data/english_to_latex.csv

--2024-10-31 00:43:31--  https://github.com/sinanuozdemir/quick-start-guide-to-llms/raw/refs/heads/main/data/english_to_latex.csv
Resolving github.com (github.com)... 20.205.243.166
Connecting to github.com (github.com)|20.205.243.166|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://media.githubusercontent.com/media/sinanuozdemir/quick-start-guide-to-llms/refs/heads/main/data/english_to_latex.csv [following]
--2024-10-31 00:43:32--  https://media.githubusercontent.com/media/sinanuozdemir/quick-start-guide-to-llms/refs/heads/main/data/english_to_latex.csv
Resolving media.githubusercontent.com (media.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to media.githubusercontent.com (media.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2085 (2.0K) [text/plain]
Saving to: ‘english_to_latex.csv’

english_to_latex.cs 100%[===================>]   2.04

In [16]:
%%capture
!pip install pip3-autoremove
!pip-autoremove torch torchvision torchaudio -y
!pip install torch --index-url https://download.pytorch.org/whl/cu121

In [32]:
import torch
torch.__version__

'2.5.0+cu121'

In [33]:
import datasets
datasets.__version__

'3.0.2'

In [17]:
import pandas as pd

In [18]:
data_path = "/content/english_to_latex.csv"
data = pd.read_csv(data_path)
data.head()

,English,LaTeX
0,integral from a to b of x squared,"\int_{a}^{b} x^2 \,dx"
1,integral from negative 1 to 1 of x squared,"\int_{-1}^{1} x^2 \,dx"
2,integral from negative 1 to infinity of x cubed,"\int_{-1}^{\inf} x^3 \,dx"
3,integral from 0 to infinity of x squared,"\int_{0}^{\inf} x^2 \,dx"
4,integral from 0 to infinity of y squared,"\int_{0}^{\inf} y^2 \,dy"


In [19]:
# Add our singular prompt
CONVERSION_PROMPT = 'Convert English to LaTeX\n'  # LaTeX conversion task

CONVERSION_TOKEN = 'LaTeX:'


# This is our "training prompt" that we want GPT2 to recognize and learn
training_examples = f'{CONVERSION_PROMPT}English: ' + data['English'] + '\n' + CONVERSION_TOKEN + ' ' + data['LaTeX'].astype(str)

print(training_examples[0])

Convert English to LaTeX
English: integral from a to b of x squared
LaTeX: \int_{a}^{b} x^2 \,dx


In [20]:
task_df = pd.DataFrame({'text': training_examples})

task_df.head(2)

,text
0,Convert English to LaTeX\nEnglish: integral fr...
1,Convert English to LaTeX\nEnglish: integral fr...


In [21]:
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    pipeline,
    Trainer,
    TrainingArguments,
    DataCollatorForLanguageModeling)

In [22]:
!pip install -q datasets

In [23]:
from datasets import Dataset

In [24]:
model = "gpt2"
tokenizer = AutoTokenizer.from_pretrained(model)
model = AutoModelForCausalLM.from_pretrained(model)

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [25]:
tokenizer.eos_token

'<|endoftext|>'

In [26]:
task_df['text'] = task_df['text'].map(lambda x: f'{x}{tokenizer.eos_token}')

In [27]:
latex_data = Dataset.from_pandas(task_df)  # turn a pandas DataFrame into a Dataset

def preprocess(examples):
    # tokenize our text but don't pad because our collator will pad for us dynamically
    return tokenizer(examples['text'], truncation=True)

latex_data = latex_data.map(preprocess, batched=True)

latex_data = latex_data.train_test_split(train_size=.8)

Map:   0%|          | 0/50 [00:00<?, ? examples/s]

In [28]:
latex_data['train'][0]

{'text': 'Convert English to LaTeX\nEnglish: 2 pi\nLaTeX: 2 * \\pi<|endoftext|>',
 'input_ids': [3103,
  1851,
  3594,
  284,
  4689,
  49568,
  198,
  15823,
  25,
  362,
  31028,
  198,
  14772,
  49568,
  25,
  362,
  1635,
  3467,
  14415,
  50256],
 'attention_mask': [1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1]}

In [29]:
# standard data collator for auto-regressive language modelling
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

In [39]:
tokenizer.pad_token = tokenizer.eos_token  # set the pad token to avoid a warning

In [40]:
training_args = TrainingArguments(
    output_dir="./english_to_latex",
    overwrite_output_dir=True, # overwrite the content of the output directory
    num_train_epochs=5, # number of training epochs
    per_device_train_batch_size=1, # batch size for training
    per_device_eval_batch_size=20,  # batch size for evaluation
    load_best_model_at_end=True,
    logging_steps=5,
    log_level='info',
    evaluation_strategy='epoch',
    save_strategy='epoch',
    # use_mps_device=True
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=latex_data["train"],
    eval_dataset=latex_data["test"],
    data_collator=data_collator,
)

trainer.evaluate()

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).

***** Running Evaluation *****
  Num examples = 10
  Batch size = 20
The following columns in the evaluation set don't have a corresponding argument in `GPT2LMHeadModel.forward` and have been ignored: text. If text are not expected by `GPT2LMHeadModel.forward`,  you can safely ignore this message.


Automatic Weights & Biases logging enabled, to disable set os.environ["WANDB_DISABLED"] = "true"
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend. Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


{'eval_loss': 4.27951717376709,
 'eval_model_preparation_time': 0.0025,
 'eval_runtime': 1.3654,
 'eval_samples_per_second': 7.324,
 'eval_steps_per_second': 0.732}

In [41]:
trainer.train()

***** Running training *****
  Num examples = 40
  Num Epochs = 5
  Instantaneous batch size per device = 1
  Total train batch size (w. parallel, distributed & accumulation) = 1
  Gradient Accumulation steps = 1
  Total optimization steps = 200
  Number of trainable parameters = 124,439,808
The following columns in the training set don't have a corresponding argument in `GPT2LMHeadModel.forward` and have been ignored: text. If text are not expected by `GPT2LMHeadModel.forward`,  you can safely ignore this message.


Epoch,Training Loss,Validation Loss,Model Preparation Time
1,1.249700,0.932448,0.002500
2,0.456600,0.649195,0.002500
3,0.618400,0.614114,0.002500
4,0.510300,0.574884,0.002500
5,0.471000,0.575440,0.002500



***** Running Evaluation *****
  Num examples = 10
  Batch size = 20
The following columns in the evaluation set don't have a corresponding argument in `GPT2LMHeadModel.forward` and have been ignored: text. If text are not expected by `GPT2LMHeadModel.forward`,  you can safely ignore this message.


Saving model checkpoint to ./english_to_latex/checkpoint-40
Configuration saved in ./english_to_latex/checkpoint-40/config.json
Configuration saved in ./english_to_latex/checkpoint-40/generation_config.json
Model weights saved in ./english_to_latex/checkpoint-40/model.safetensors

***** Running Evaluation *****
  Num examples = 10
  Batch size = 20
The following columns in the evaluation set don't have a corresponding argument in `GPT2LMHeadModel.forward` and have been ignored: text. If text are not expected by `GPT2LMHeadModel.forward`,  you can safely ignore this message.
Saving model checkpoint to ./english_to_latex/checkpoint-80
Configuration saved in ./english_to_latex/checkpoint-80/config.json
Configuration saved in ./english_to_latex/checkpoint-80/generation_config.json
Model weights saved in ./english_to_latex/checkpoint-80/model.safetensors

***** Running Evaluation *****
  Num examples = 10
  Batch size = 20
The following columns in the evaluation set don't have a correspondi

TrainOutput(global_step=200, training_loss=0.7993447506427764, metrics={'train_runtime': 204.6157, 'train_samples_per_second': 0.977, 'train_steps_per_second': 0.977, 'total_flos': 3136014720000.0, 'train_loss': 0.7993447506427764, 'epoch': 5.0})

In [42]:
trainer.save_model()

Saving model checkpoint to ./english_to_latex
Configuration saved in ./english_to_latex/config.json
Configuration saved in ./english_to_latex/generation_config.json
Model weights saved in ./english_to_latex/model.safetensors


In [ ]:
# load up our gpt pre-trained on latex cheat sheets
math_latex_gpt2 = AutoModelForCausalLM.from_pretrained('./english_to_latex')

In [64]:
text = """Convert English to LaTeX
English: x squared
"""

In [72]:
training_examples[0]

'Convert English to LaTeX\nEnglish: integral from a to b of x squared\nLaTeX: \\int_{a}^{b} x^2 \\,dx'

In [68]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
inputs = tokenizer.encode(text, return_tensors='pt').to(device)

In [69]:
outputs = model.generate(inputs, max_new_tokens=200)
outputs

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


tensor([[ 3103,  1851,  3594,   284,  4689, 49568,   198, 15823,    25,  2124,
         44345,   198, 14772, 49568,    25,  3467, 31944,    90,    87,    61,
            17, 18477,    87,    61,    17,    92,  2124,    61,    17,  3467,
            11, 34350,    61,    17,  3467,    11, 34350,    61,    17,  3467,
            11, 34350,    61,    17,  3467,    11, 34350,    61,    17,  3467,
            11, 34350,    61,    17,  3467,    11, 34350,    61,    17,  3467,
            11, 34350,    61,    17,  3467,    11, 34350,    61,    17,  3467,
            11, 34350,    61,    17,  3467,    11, 34350,    61,    17,  3467,
            11, 34350,    61,    17,  3467,    11, 34350,    61,    17,  3467,
            11, 34350,    61,    17,  3467,    11, 34350,    61,    17,  3467,
            11, 34350,    61,    17,  3467,    11, 34350,    61,    17,  3467,
            11, 34350,    61,    17,  3467,    11, 34350,    61,    17,  3467,
            11, 34350,    61,    17,  3467,    11, 3

In [70]:
response = tokenizer.decode(outputs[0], skip_special_tokens=True)

In [71]:
response

'Convert English to LaTeX\nEnglish: x squared\nLaTeX: \\frac{x^2}{x^2} x^2 \\,dx^2 \\,dx^2 \\,dx^2 \\,dx^2 \\,dx^2 \\,dx^2 \\,dx^2 \\,dx^2 \\,dx^2 \\,dx^2 \\,dx^2 \\,dx^2 \\,dx^2 \\,dx^2 \\,dx^2 \\,dx^2 \\,dx^2 \\,dx^2 \\,dx^2 \\,dx^2 \\,dx^2 \\,dx^2 \\,dx^2 \\,dx^2 \\,dx^2 \\,dx^2 \\,dx^2 \\,dx^2 \\,dx^2 \\,dx^2 \\,dx^2 \\,dx^2 \\,dx^2 \\,dx^2 \\,dx^2 \\,dx^2 \\,dx'